In [1]:
import os
import sys
from importlib import reload
import random

from tqdm import tqdm_notebook
import torch as T
import torch.nn as nn
import numpy as np

sys.path.append('../')
from torch.utils.data import DataLoader

from args import define_main_parser

from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

from data.card import TransactionDataset
from models import CLTrial_Bert
from models import tabformer_bert
from scripts.utils import random_split_dataset
import data.datacollator as datacoll

In [2]:
reload(CLTrial_Bert)

<module 'models.CLTrial_Bert' from '/Users/murugesan.vadivel/DEV/works/Clinical-Transformer/notebook/../models/CLTrial_Bert.py'>

In [3]:
from argparse import Namespace
config = vars(Namespace(cached=False, checkpoint=0, data_extension='', data_fname='card_transaction.v2', data_root='./data/credit_card/', data_type='card', do_eval=False, do_train=True, field_ce=True, field_hs=64, flatten=False, jid=1, lm_type='bert', log_dir='sam/logs', mlm=True, mlm_prob=0.15, nrows=None, num_train_epochs=3, output_dir='sam', save_steps=500, seed=9, skip_user=False, stride=5, user_ids=None, vocab_file='vocab.nb'))
config['data_root'] = "../dataset/credit_card/"
config['output_dir'] = "sample"
config['log_dir'] = "sample/logs"
os.makedirs(config['output_dir'], exist_ok=True)
os.makedirs(config['log_dir'], exist_ok=True)

In [4]:
seed = config['seed']
random.seed(seed)  # python
np.random.seed(seed)  # numpy
T.manual_seed(seed)  # torch
if T.cuda.is_available():
    T.cuda.manual_seed_all(seed)  # torch.cuda

In [5]:
dataset = TransactionDataset(root=config['data_root'],
                            fname=config['data_fname'],
                            fextension="",
                            vocab_dir=config['output_dir'],
                            nrows=None,
                            user_ids=None,
                            seq_len=20,
                            mlm=True,
                            cached=config['cached'],
                            stride=10,
                            flatten=config['flatten'],
                            return_labels=True,
                            skip_user=True)

100%|██████████| 20/20 [00:02<00:00,  8.46it/s]


In [6]:
vocab = dataset.vocab
custom_special_tokens = vocab.get_special_tokens()

totalN = len(dataset)
totalN = len(dataset)
trainN = int(0.6 * totalN)

valtestN = totalN - trainN
valN = int(valtestN * 0.5)
testN = valtestN - valN
lengths = [trainN, valN, testN]
print(f"# lengths: train [{trainN}]  valid [{valN}]  test [{testN}]")
print("# lengths: train [{:.2f}]  valid [{:.2f}]  test [{:.2f}]".format(trainN / totalN, valN / totalN,
                                                                               testN / totalN))

# lengths: train [25447]  valid [8483]  test [8483]
# lengths: train [0.60]  valid [0.20]  test [0.20]


In [7]:
train_dataset, eval_dataset, test_dataset = random_split_dataset(dataset, lengths)

In [8]:
tab_net = CLTrial_Bert.ClTrial(custom_special_tokens,
                                  vocab=vocab,
                                  field_ce=config['field_ce'],
                                  flatten=config['flatten'],
                                  ncols=dataset.ncols,
                                  field_hidden_size=config['field_hs']
                                  )

In [9]:
collactor_cls = "TransDataCollatorForLanguageModeling"
data_collator = datacoll.TransDataCollatorForLanguageModeling(
        tokenizer=tab_net.tokenizer, mlm=True, mlm_probability=config['mlm_prob']
    )

In [10]:
train_dataloader = DataLoader(
            train_dataset,
            batch_size=10,
            collate_fn=data_collator)

In [11]:
model = tab_net.model
optim_params = {'betas': (0.9, 0.999), 'eps': 1e-08, 'lr': 5e-05}
optim = T.optim.AdamW(model.parameters(), **optim_params)

In [29]:
total_loss = 0
for inps in tqdm_notebook(train_dataloader):
    print(inps.keys())
    print(inps['Ouput'])
    #print(inps['Ouput'][:,:, -1])
"""
    #print(inps.keys())
    #print(inps['input_ids'].shape)
    #print(inps['masked_lm_labels'].shape)
    #print(inps['masked_lm_labels'], )
    optim.zero_grad()
    #print(inps['input_ids'].shape)
    labels = inps.pop("Ouput")
    model.train()
    inps['input_ids'] = inps['input_ids']
    inps['masked_lm_labels'] = inps['masked_lm_labels']
    outputs =model(**inps)
    print('out',outputs)
    break
    loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
    print(loss)
    loss.backward()
    optim.step()
    total_loss += loss.item()
"""

/var/folders/r7/xqg2bpnj0bj0pdv44sb5k8m40000gp/T/ipykernel_71512/311210400.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inps in tqdm_notebook(train_dataloader):


  0%|          | 0/2545 [00:00<?, ?it/s]

/Users/murugesan.vadivel/DEV/works/Clinical-Transformer/notebook/../data/datacollator.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labs = [torch.tensor(e[1], dtype=torch.long) for e in examples]
/Users/murugesan.vadivel/DEV/works/Clinical-Transformer/notebook/../data/datacollator.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  examples = [torch.tensor(e[0], dtype=torch.long) for e in examples]


dict_keys(['input_ids', 'masked_lm_labels', 'Ouput'])
[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]
dict_keys(['input_ids', 'masked_lm_labels', 'Ouput'])
[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

'\n    #print(inps.keys())\n    #print(inps[\'input_ids\'].shape)\n    #print(inps[\'masked_lm_labels\'].shape)\n    #print(inps[\'masked_lm_labels\'], )\n    optim.zero_grad()\n    #print(inps[\'input_ids\'].shape)\n    labels = inps.pop("Ouput")\n    model.train()\n    inps[\'input_ids\'] = inps[\'input_ids\']\n    inps[\'masked_lm_labels\'] = inps[\'masked_lm_labels\']\n    outputs =model(**inps)\n    print(\'out\',outputs)\n    break\n    loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]\n    print(loss)\n    loss.backward()\n    optim.step()\n    total_loss += loss.item()\n'

In [13]:
vocab.get_field_keys()

['Card',
 'Timestamp',
 'Amount',
 'Use Chip',
 'Merchant Name',
 'Merchant City',
 'Merchant State',
 'Zip',
 'MCC',
 'Errors?',
 'SPECIAL']